In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Load the dataset
file_path = '/kaggle/input/dengue/Dengue.csv'
dengue_data = pd.read_csv(file_path)

In [3]:
# Print the column names of the DataFrame
print(dengue_data.columns)

Index(['Age', 'Sex', 'Month Affected', 'Residence Address',
       'Duration of fever (days)', 'Current body temperature (°C)',
       'White Blood Cell (WBC) Count', 'Back pain in eyes', 'Joint pain',
       'Muscle pain', 'Vomiting', 'Hemoglobin Range', 'Hematocrit Range',
       'Platelets Count', 'Shock (Low/Medium/High)', 'Rash',
       'Pleural effusion', 'Ascites', 'Bleeding', 'Slow heart rate', 'Igg',
       'Igm', 'NS1', 'Travelling history', 'Headache'],
      dtype='object')


In [4]:
from transformers import BertTokenizer
import numpy as np

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example column that contains text data
text_column = 'NS1'  # Replace with the name of your text column
max_length = 128  # Define the max length for BERT input

# Tokenize and encode the text data
encoded_texts = tokenizer.batch_encode_plus(
    dengue_data[text_column].tolist(),
    add_special_tokens=True,
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='np'
)

input_ids = encoded_texts['input_ids']
attention_masks = encoded_texts['attention_mask']


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
import pandas as pd

text_columns = dengue_data.select_dtypes(include=['object']).columns.tolist()

# Display the text columns
text_columns


['Sex',
 'Month Affected',
 'Residence Address',
 'Back pain in eyes',
 'Joint pain',
 'Muscle pain',
 'Vomiting',
 'Hemoglobin Range',
 'Hematocrit Range',
 'Shock (Low/Medium/High)',
 'Rash',
 'Pleural effusion',
 'Ascites',
 'Bleeding',
 'Slow heart rate',
 'Igg',
 'Igm',
 'NS1',
 'Travelling history',
 'Headache']

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
dengue_data = pd.read_csv('/kaggle/input/dengue/Dengue.csv')  # Replace with the correct path

# Define columns
numerical_cols = ["Age", "Duration of fever (days)", "Current body temperature (°C)", 
                  "White Blood Cell (WBC) Count", "Platelets Count"]
categorical_cols = ['NS1']
Text_columns = ['Sex', 'Month Affected', 'Residence Address', 'Back pain in eyes', 
                'Joint pain', 'Muscle pain', 'Vomiting', 'Hemoglobin Range', 
                'Hematocrit Range', 'Shock (Low/Medium/High)', 'Rash', 'Pleural effusion', 
                'Ascites', 'Bleeding', 'Slow heart rate', 'Igg', 'Igm', 'Travelling history', 'Headache']

# Preprocessing steps
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Combine the lists of columns to drop
columns_to_drop = Text_columns

# Apply the preprocessing
X_processed = preprocessor.fit_transform(dengue_data.drop(columns=columns_to_drop, axis=1))

# Target variable
y = dengue_data['NS1']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

NameError: name 'StandardScaler' is not defined

In [ ]:
# from tensorflow.keras.utils import to_categorical

# # Assuming 'NS1' is your target variable
# target = dengue_data['NS1'].values
# # If the target is categorical, convert it to one-hot encoding
# target_encoded = to_categorical(target)


In [ ]:
# from sklearn.model_selection import train_test_split

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X_processed, target_encoded, test_size=0.2, random_state=42)


In [ ]:
# from tensorflow.keras.utils import to_categorical

# # Assuming 'NS1' is your target variable
# target = dengue_data['NS1'].values
# # If the target is categorical, convert it to one-hot encoding
# target_encoded = to_categorical(target)


In [ ]:
# Calculate the input shape for non-text data
input_shape_non_text = X_processed.shape[1]  # Number of features in your preprocessed dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, Concatenate, Add, Reshape
from transformers import TFBertModel
from tensorflow.keras.optimizers import Adam

# Load pre-trained BERT model
bert = TFBertModel.from_pretrained('bert-base-uncased')

from tensorflow.keras.layers import Dense

def residual_lstm_block(x, units):
    # LSTM layer
    lstm_out = LSTM(units=units, return_sequences=True)(x)
    
    # Shape transformation to match the LSTM output to the input
    # You can adjust the units to match the shape of 'x'
    x_transformed = Dense(units, activation='linear')(x)

    # Adding a residual connection
    x = Add()([x_transformed, lstm_out])
    return x

# Your existing model code...


embedding_dim = 128

# Assuming your text data is in a column named 'text_column'
max_length = 128  # Define the max length based on your text data
text_input = Input(shape=(max_length,), dtype='int32', name='text_input')
bert_output = bert(text_input)[0]  # Use the sequence output of BERT


from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K


expanded_embedding_output = Lambda(lambda x: K.repeat(x, max_length))(flattened_embedding_output)
combined = Concatenate(axis=-1)([bert_output, expanded_embedding_output])


total_number_of_unique_categories = sum(dengue_data[categorical_col].nunique() for categorical_col in categorical_cols)
vocab_size = total_number_of_unique_categories

input_shape_non_text = 10  # Adjust this based on your non-text data
other_input = Input(shape=(input_shape_non_text,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_shape_non_text)(other_input)

# Flatten the embedding output
flattened_embedding_output = Flatten()(embedding_layer)
expanded_embedding_output = Lambda(lambda x: K.repeat(x, max_length))(flattened_embedding_output)
combined = Concatenate(axis=-1)([bert_output, expanded_embedding_output])
#combined = Concatenate()([bert_output, flattened_embedding_output])


for _ in range(19):
    combined = residual_lstm_block(combined, 64)

# Add Dense layers
for _ in range(19):
    combined = Dense(units=64, activation='relu')(combined)

# Output layer
num_classes = 2  # Adjust based on your target classes
output = Dense(num_classes, activation='softmax')(combined)

# Build the model
model = Model(inputs=[text_input, other_input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
dengue_data = pd.read_csv('/kaggle/input/dengue/Dengue.csv')  # Replace with the correct path

# Define columns
numerical_cols = ["Age", "Duration of fever (days)", "Current body temperature (°C)", 
                  "White Blood Cell (WBC) Count", "Platelets Count"]
categorical_cols = ['NS1']
Text_columns = ['Sex', 'Month Affected', 'Residence Address', 'Back pain in eyes', 
                'Joint pain', 'Muscle pain', 'Vomiting', 'Hemoglobin Range', 
                'Hematocrit Range', 'Shock (Low/Medium/High)', 'Rash', 'Pleural effusion', 
                'Ascites', 'Bleeding', 'Slow heart rate', 'Igg', 'Igm', 'Travelling history', 'Headache']

# Preprocessing steps
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Combine the lists of columns to drop
columns_to_drop = Text_columns

# Apply the preprocessing
X_non_text = preprocessor.fit_transform(dengue_data.drop(columns=columns_to_drop, axis=1))

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

texts = Text_columns  
encoded_texts = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors='tf')
X_train_text = encoded_texts['input_ids']

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
X_train_text, X_test_text, X_train_other, X_test_other, y_train, y_test = train_test_split(
    X_train_text, 
    X_non_text, 
    y, 
    test_size=0.2, 
    random_state=42
)


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
# Evaluate the model
# loss, accuracy = model.evaluate([X_test_text, X_test_other], y_test)
# print(f"Test Accuracy: {accuracy*100:.2f}%")
